In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time

In [2]:
class BookSelection:
    if __name__ == "__main__":
        def __init__(self, url, search_keywords):
            self.url = url
            self.search_keywords = search_keywords

        def set_drivers(self):
            try:
                service = Service(executable_path=ChromeDriverManager().install())
                options = Options()
                # 참고 = https://codingstep.tistory.com/entry/%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%85%80%EB%A0%88%EB%8B%88%EC%9B%80-%ED%81%AC%EB%A1%AC-%EC%98%B5%EC%85%98-python-selenium-chrome-optionsFeat-headless-window-size-user-agent-etc
                #지정한 user-agent로 설정합니다.
#                 user_agent = "Mozilla/5.0 (Linux; Android 9; SM-G975F) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.83 Mobile Safari/537.36"
#                 options.add_argument('user-agent=' + user_agent)
#                 options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
#                 options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
#                 options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
#                 options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
#                 options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
#                 options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
#                 options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
                driver = webdriver.Chrome(service=service, options=options)  ##버전에 상관없이 현재 OS에 설치된 크롬브라우저 사용
                return driver
            except Exception as e:
                print(e)
        
        def get_url(self, driver):
            try:
                driver.get(self.url)
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
                
        def get_searchKeywords(self, driver):
            try:
                search_input = driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/div/input")
                search_input.send_keys(self.search_keywords)
                driver.find_element(By.XPATH, "/html/body/div[4]/div[1]/div[1]/div[1]/form[2]/div/input").click()
                driver.implicitly_wait(time_to_wait=10)
            except Exception as e:
                print(e)
                driver.quit()
            
    else:
        exit()

In [3]:
bookSelection = BookSelection("http://www.kyobobook.co.kr/", "python")
driver = bookSelection.set_drivers()
bookSelection.get_url(driver)
bookSelection.get_searchKeywords(driver)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [/Users/administrator/.wdm/drivers/chromedriver/mac64/97.0.4692.71/chromedriver] found in cache


In [4]:
searchTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[1]/ul").text.split('\n')
subTabList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[2]/ul").text.split('\n')
categoryList = driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[3]/table/tbody/tr[1]/td/ul").text.split('\n')

In [5]:
print(searchTabList)
print(subTabList)
print(categoryList)

['상품(5,290)', '중고장터(515)', '스토리K(25)', '리뷰(1,262)', '책소개.목차(3,400)']
['전체 5,290', '국내도서 1,157', '외국도서 3,596', 'eBook 500', 'sam 90', '음반/영상 37', '기프트 0']
['컴퓨터/IT 1,111', '어린이(초등) 11', '경제/경영 11', '과학 7', '청소년 3', '정치/사회 3', '취업/수험서 2', '자기계발 2']


In [6]:
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]").click()
driver.implicitly_wait(3)

In [7]:
driver.find_element(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[6]/div[2]/span[2]/ul/li[3]").click()
driver.implicitly_wait(10)

In [8]:
result = driver.find_element(By.ID, "search_list")
time.sleep(3)

## 내부 구조 파악 필요

In [9]:
result_length = len(result.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a"))
list_count = 0

while (list_count < result_length):
    titleLinks = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[3]/div[8]/form[1]/table/tbody/tr/td[2]/div[2]/a")
    titleLinks[list_count].click()
    time.sleep(2)

    detailInfo = dict()
    # class="box_detail_point"
    detailPoints = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[1]/*")
    for detail in detailPoints:
        detailInfo[detail.get_attribute("class")] = detail.text
        
    # class="book_keyword"
    keywords = []
    if len(driver.find_elements(By.CLASS_NAME, "book_keyword")) > 0:
        for keyword in driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[2]/form/div[2]/div[4]/div[2]/*"):
            keywords.append(keyword.text)
        detailInfo['book_keyword'] = ", ".join(keywords)

    # "content"
    detailcontentList = []
    contents = driver.find_elements(By.XPATH, "/html/body/div[2]/div[1]/div[2]/div[5]/div[1]/div[3]/*")
    for content in contents:
        if len(content.find_elements(By.TAG_NAME, "a"))>0:
#             print("a Tag cnt = "+str(len(content.find_elements(By.TAG_NAME, "a"))))
            num = 0
            for aTag in content.find_elements(By.TAG_NAME, "a"):
#                 print("text = " + aTag.text + " Class_name = " + aTag.get_attribute("class"))
#                 print(num)
                num+=1
                if aTag.get_attribute("class") == "btn_small btn_more" and aTag.text == "더보기":
                    aTag.click()
#                     print("click")
    for content in contents:
            detailcontentList.append(content.text)
    detailInfo['content'] = "\n".join(detailcontentList)
    print(detailInfo)
    driver.back()
    list_count+=1
    time.sleep(2)

{'icon': '', 'title': '혼자 공부하는 파이썬 파이썬 최신 버전 반영', 'author': '윤인성 지음 | 한빛미디어 | 2019년 06월 10일 출간', 'review': 'Klover\n총 4 중\n4 9.6 (리뷰 216개)\n클로버 리뷰쓰기', 'rank': '국내도서 주간베스트 242위 | 컴퓨터/IT 주간베스트 10위 교보문고 베스트셀러', 'book_keyword': '프로그래밍, 딕셔너리, 반 복문, 메소드, 리스트, 자습서', 'content': '도서·상품정보 1:1 문의하기\nISBN 9791162241882(1162241888)\n쪽수 460쪽\n크기 188 * 258 * 28 mm /1001g 판형알림\n책소개\n이 책이 속한 분야\n컴퓨터/IT > 프로그래밍 언어 > Python(파이썬)\n컴퓨터/IT > 대학교재\n대학교재 > 컴퓨터\n이 책의 주제어\n#프로그래밍 #자습서 #입문자 #손코딩 #전공자\n\n혼자 해도 충분하다! 1:1 과외하듯 배우는 파이썬 프로그래밍 자습서(파이썬 최신 버전 반영)\n27명의 베타리더 검증으로, ‘함께 만든’ 입문자 맞춤형 도서\n이 책은 독학으로 프로그래밍 언어를 처음 배우려는 입문자가, 혹은 파이썬을 배우려는 입문자가 ‘꼭 필요한 내용을 제대로’ 학습할 수 있도록 구성했다. ‘무엇을’, ‘어떻게’ 학습해야 할지조차 모르는 입문자의 막연한 마음을 살펴, 과외 선생님이 알려주듯 친절하게, 그러나 핵심적인 내용만 콕콕 집어준다. 책의 첫 페이지를 펼쳐서 마지막 페이지를 덮을 때까지, 혼자서도 충분히 파이썬을 배울 수 있다는 자신감과 확신이 계속될 것이다!\n\n27명의 베타리더와 함께 구성하여 입문자에게 맞는 난이도, 분량, 학습 요소 등을 적극 반영했다. 어려운 용어와 개념은 한번 더 풀어 쓰고, 복잡한 설명은 눈에 잘 들어오는 그림으로 풀어 냈다. ‘혼자 공부해본’ 여러 입문자의 초심과 눈높이가 책 곳곳에 반영된 것이 이 책의 가장 큰 장점이다.\n상세이미지\n\n저자소개\n저자 

ElementClickInterceptedException: Message: element click intercepted: Element <a href="#" class="btn_small btn_more">...</a> is not clickable at point (780, 14). Other element would receive the click: <a href="#guide">...</a>
  (Session info: chrome=97.0.4692.99)
Stacktrace:
0   chromedriver                        0x000000010145de69 chromedriver + 5160553
1   chromedriver                        0x00000001013e8593 chromedriver + 4679059
2   chromedriver                        0x0000000100f9c2c8 chromedriver + 172744
3   chromedriver                        0x0000000100fd8f6b chromedriver + 421739
4   chromedriver                        0x0000000100fd6baf chromedriver + 412591
5   chromedriver                        0x0000000100fd4350 chromedriver + 402256
6   chromedriver                        0x0000000100fd316b chromedriver + 397675
7   chromedriver                        0x0000000100fc6d19 chromedriver + 347417
8   chromedriver                        0x0000000100fef362 chromedriver + 

In [ ]:
# detailInfo

In [ ]:
# driver.back()

In [ ]:
# driver.quit()